# Gradio Interface With LLMs

Today, I built user interfaces using the outrageously simple Gradio framework.



In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr 
import openai

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCu


In [3]:
# Class to handle API communication with OpenAI
class OpenAICommunicator:
    def __init__(self, openai_instance, model):
        self.openai = openai_instance
        self.model = model

    def get_response(self, system_prompt, user_prompt):
        try:
            response = self.openai.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            response_content = response.choices[0].message.content
            if not response_content:
                print("Error: Empty response from OpenAI")
                return "{}"  # Return empty JSON structure
            return response_content
        except Exception as e:
            print(f"Error fetching response from OpenAI: {e}")
            return "{}"  # Return empty JSON structure in case of error


In [4]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [5]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [6]:
message_gpt("What is today's date?")

"Today's date is October 5, 2023."

In [7]:
#Extracting Data From Website

class Website:
    def __init__(self, url):
        self.url = url
        self.body = self._fetch_website_content()
        self.title, self.text, self.links = self._parse_content()

    def _fetch_website_content(self):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
        }
        response = requests.get(self.url, headers=headers)
        return response.content

    def _parse_content(self):
        soup = BeautifulSoup(self.body, 'html.parser')
        title = soup.title.string if soup.title else "No title found"
        text = ""
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            text = soup.body.get_text(separator="\n", strip=True)
        links = [link.get('href') for link in soup.find_all('a') if link.get('href')]
        return title, text, links

    def get_contents(self):
        return f"## Webpage Title:\n{self.title}\n\n## Webpage Contents:\n{self.text}\n\n"


In [8]:
# Class for Brochure creation logic
class BrochureCreator:
    def __init__(self, company_name, url, openai_communicator):
        self.company_name = company_name
        self.url = url
        self.openai_communicator = openai_communicator

    def _get_links_user_prompt(self, website):
        user_prompt = f"Here is the list of links on the website of {website.url} - "
        user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, or email links.\n"
        user_prompt += "Links (some might be relative links):\n"
        user_prompt += "\n".join(website.links)
        return user_prompt

    def _get_links(self, website):
        link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
        link_system_prompt += "You should respond in JSON format as shown in the example below:\n"
        link_system_prompt += """
        {
            "links": [
                {"type": "about page", "url": "https://full.url/goes/here/about"},
                {"type": "careers page", "url": "https://another.full.url/careers"}
            ]
        }
        """
        user_prompt = self._get_links_user_prompt(website)
        response_content = self.openai_communicator.get_response(link_system_prompt, user_prompt)
        try:
            return json.loads(response_content)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return {"links": []}  # Return empty links in case of error

    def _get_all_details(self):
        website = Website(self.url)
        result = f"## Landing page:\n{website.get_contents()}"
        links = self._get_links(website)
        print("Found links:", links)
        for link in links["links"]:
            result += f"\n\n## {link['type']}:\n"
            result += Website(link["url"]).get_contents()
        return result

    def _get_brochure_user_prompt(self):
        user_prompt = f"You are looking at a company called: {self.company_name}\n"
        user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
        user_prompt += self._get_all_details()
        user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
        return user_prompt

    def create_brochure(self):
        system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors, and recruits. Respond in markdown.\
Include details of company culture, customers, and careers/jobs if you have the information."
        user_prompt = self._get_brochure_user_prompt()
        markdown_response = self.openai_communicator.get_response(system_prompt, user_prompt)
        return markdown_response

In [9]:
import gradio as gr
import openai
from typing import Optional, Generator
import os

class BrochureCreator:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = self.api_key
    
    def create_brochure_stream(self, url: str, user_input: str = "") -> Generator[str, None, None]:
        """
        Creates a brochure using OpenAI's ChatCompletion API with streaming
        """
        company_name = url.split("://")[-1].split(".")[0].capitalize()
        
        messages = [
            {
                "role": "system",
                "content": """You are a professional brochure writer. Create engaging, well-structured brochures 
                with markdown formatting. Use proper markdown syntax for headings (##), bullet points (*), 
                emphasis (**bold**), and other formatting elements."""
            },
            {
                "role": "user",
                "content": f"""Create a professional brochure for {company_name} (URL: {url}).
                
                Structure the brochure with these sections using proper markdown formatting:
                
                1. Company Overview
                2. Key Products/Services
                3. Value Proposition
                4. Why Choose Us
                5. Contact Information
                
                Additional requirements: {user_input}
                
                Use markdown features:
                - ## for section headings
                - * for bullet points
                - ** for bold text
                - > for notable quotes or highlights
                - --- for section separators
                """
            }
        ]
        
        try:
            stream = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=1500,
                temperature=0.7,
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0,
                stream=True
            )
            
            accumulated_text = ""
            
            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    delta_content = chunk.choices[0].delta.content
                    accumulated_text += delta_content
                    yield accumulated_text
                    
        except Exception as e:
            yield f"### Error\n\nFailed to generate brochure content: {str(e)}"

def generate_brochure_response(url, additional_requirements):
    """
    Generator function for streaming brochure generation
    """
    try:
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            yield "### Error\n\nOpenAI API key not found. Please set the OPENAI_API_KEY environment variable."
            return
        
        brochure_creator = BrochureCreator(api_key)
        
        for content in brochure_creator.create_brochure_stream(
            url=url,
            user_input=additional_requirements
        ):
            yield content
            
    except Exception as e:
        yield f"### Error\n\nError generating brochure: {str(e)}"

# Create Gradio interface with streaming markdown
interface = gr.Interface(
    fn=generate_brochure_response,
    inputs=[
        gr.Textbox(
            label="Website URL:",
            placeholder="Enter the website URL (e.g., https://www.example.com)",
            lines=1
        ),
        gr.Textbox(
            label="Additional Requirements:",
            placeholder="Enter any specific requirements for the brochure...",
            lines=4
        )
    ],
    outputs=[
        gr.Markdown(
            label="Generated Brochure",
            elem_classes="markdown-output"
        )
    ],
    title="Streaming Brochure Generator",
    description="Generate a custom brochure by providing a website URL and your specific requirements. Watch as the content streams in real-time with proper markdown formatting!",
    css="""
        .markdown-output {
            background-color: white;
            padding: 20px;
            border-radius: 8px;
            border: 1px solid #ddd;
            max-height: 600px;
            overflow-y: auto;
        }
        .markdown-output h2 {
            margin-top: 20px;
            margin-bottom: 10px;
            color: #2c3e50;
        }
        .markdown-output ul {
            margin-left: 20px;
        }
        .markdown-output blockquote {
            border-left: 4px solid #3498db;
            margin: 10px 0;
            padding: 10px 20px;
            background-color: #f8f9fa;
        }
    """,
    flagging_mode="never"
)

if __name__ == "__main__":
    interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
